# Women's Clothing Recommendation Prediction with Interpretable AI

**Student Name:** Georgios Kitsakis  
**Course:** Advanced Customer Analytics  
**Assignment:** Interpretable Predictions - 2025

---

## Dataset

**Source:** Women's E-Commerce Clothing Reviews  
**Link:** https://www.kaggle.com/datasets/nicapotato/womens-ecommerce-clothing-reviews  
**Size:** 23,486 customer reviews

**Prediction Task:** Predict if a customer will recommend a clothing item and provide actionable insights on what to change.

**Approach:** I will use Logistic Regression to predict customer recommendations and generate counterfactual explanations to show what changes would improve recommendations.

---

## 1. Install Packages and Import Libraries

In [ ]:
# Install required packages
!pip install scikit-learn pandas numpy matplotlib seaborn kagglehub --quiet

print("All packages installed successfully!")

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import os
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

import kagglehub

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("All libraries imported successfully!")

## 2. Load and Explore Data

In [ ]:
# Download and load dataset
path = kagglehub.dataset_download("nicapotato/womens-ecommerce-clothing-reviews")
csv_files = [f for f in os.listdir(path) if f.endswith('.csv')]
clothing_df = pd.read_csv(os.path.join(path, csv_files[0]))

print(f"Dataset loaded: {clothing_df.shape[0]:,} reviews")
print(f"\nColumns: {clothing_df.columns.tolist()}")
print(f"\nSample data:")
clothing_df.head()

## 3. Exploratory Data Analysis

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(16, 10))

# Rating distribution
axes[0, 0].bar(clothing_df['Rating'].value_counts().sort_index().index,
               clothing_df['Rating'].value_counts().sort_index().values, color='skyblue')
axes[0, 0].set_title('Rating Distribution', fontweight='bold', fontsize=12)
axes[0, 0].set_xlabel('Rating')
axes[0, 0].set_ylabel('Count')

# Recommendation distribution
rec_counts = clothing_df['Recommended IND'].value_counts()
axes[0, 1].bar(['Not Recommended', 'Recommended'],
               [rec_counts.get(0, 0), rec_counts.get(1, 0)],
               color=['coral', 'lightgreen'])
axes[0, 1].set_title('Recommendation Distribution', fontweight='bold', fontsize=12)
axes[0, 1].set_ylabel('Count')

# Age distribution
axes[0, 2].hist(clothing_df['Age'].dropna(), bins=30, color='purple', alpha=0.7, edgecolor='black')
axes[0, 2].set_title('Customer Age Distribution', fontweight='bold', fontsize=12)
axes[0, 2].set_xlabel('Age')
axes[0, 2].set_ylabel('Count')

# Top Divisions
top_divisions = clothing_df['Division Name'].value_counts().head(5)
axes[1, 0].barh(range(len(top_divisions)), top_divisions.values, color='teal')
axes[1, 0].set_yticks(range(len(top_divisions)))
axes[1, 0].set_yticklabels(top_divisions.index)
axes[1, 0].set_title('Top 5 Divisions', fontweight='bold', fontsize=12)
axes[1, 0].set_xlabel('Count')

# Top Departments
top_depts = clothing_df['Department Name'].value_counts().head(5)
axes[1, 1].barh(range(len(top_depts)), top_depts.values, color='orange')
axes[1, 1].set_yticks(range(len(top_depts)))
axes[1, 1].set_yticklabels(top_depts.index)
axes[1, 1].set_title('Top 5 Departments', fontweight='bold', fontsize=12)
axes[1, 1].set_xlabel('Count')

# Rating vs Recommendation
rec_by_rating = clothing_df.groupby('Rating')['Recommended IND'].mean()
axes[1, 2].plot(rec_by_rating.index, rec_by_rating.values, marker='o', linewidth=2, markersize=8, color='green')
axes[1, 2].set_title('Recommendation Rate by Rating', fontweight='bold', fontsize=12)
axes[1, 2].set_xlabel('Rating')
axes[1, 2].set_ylabel('Recommendation Rate')
axes[1, 2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nKey Statistics:")
print(f"  Recommendation Rate: {clothing_df['Recommended IND'].mean()*100:.1f}%")
print(f"  Average Rating: {clothing_df['Rating'].mean():.2f}")
print(f"  Average Age: {clothing_df['Age'].mean():.1f}")

## 4. Data Preprocessing

In [ ]:
# Handle missing values
df = clothing_df.copy()
df = df.dropna(subset=['Recommended IND'])
df['Age'] = df['Age'].fillna(df['Age'].median())
df['Rating'] = df['Rating'].fillna(3)
df['Positive Feedback Count'] = df['Positive Feedback Count'].fillna(0)
df['Division Name'] = df['Division Name'].fillna('Unknown')
df['Department Name'] = df['Department Name'].fillna('Unknown')
df['Class Name'] = df['Class Name'].fillna('Unknown')
df['recommended'] = df['Recommended IND'].astype(int)

print(f"Data preprocessed: {df.shape[0]:,} reviews")
print(f"Recommendation rate: {df['recommended'].mean():.1%}")

## 5. Build Logistic Regression Model

Using Logistic Regression for its simplicity and interpretability.

In [ ]:
# Prepare features
numeric_features = ['Age', 'Rating', 'Positive Feedback Count']
categorical_features = ['Division Name', 'Department Name', 'Class Name']

X = df[numeric_features + categorical_features]
y = df['recommended']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Create preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

# Build and train Logistic Regression model
model = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(max_iter=150, random_state=42))
])

print("Training Logistic Regression model...\n")
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)[:, 1]

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Model Performance:")
print(f"  Accuracy:  {accuracy*100:.2f}%")
print(f"  Precision: {precision*100:.2f}%")
print(f"  Recall:    {recall*100:.2f}%")
print(f"  F1-Score:  {f1*100:.2f}%")

## 6. Model Performance Visualization

In [ ]:
# Confusion Matrix and Metrics
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[0], 
            xticklabels=['Not Recommend', 'Recommend'],
            yticklabels=['Not Recommend', 'Recommend'])
axes[0].set_title('Confusion Matrix', fontweight='bold', fontsize=13)
axes[0].set_ylabel('True Label', fontweight='bold')
axes[0].set_xlabel('Predicted Label', fontweight='bold')

# Performance Metrics
metrics = ['Accuracy', 'Precision', 'Recall', 'F1-Score']
values = [accuracy*100, precision*100, recall*100, f1*100]
colors = plt.cm.RdYlGn(np.linspace(0.4, 0.9, len(metrics)))
bars = axes[1].barh(metrics, values, color=colors, edgecolor='black', linewidth=2)
for bar, val in zip(bars, values):
    axes[1].text(val + 1, bar.get_y() + bar.get_height()/2, f'{val:.2f}%',
                 va='center', fontweight='bold', fontsize=11)
axes[1].set_xlabel('Score (%)', fontweight='bold', fontsize=12)
axes[1].set_title('Model Performance Metrics', fontweight='bold', fontsize=13)
axes[1].set_xlim([0, 105])
axes[1].grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nLogistic Regression achieves {accuracy*100:.2f}% accuracy")

## 7. Generate Counterfactual Explanations

Using the Logistic Regression model to show what changes would make customers recommend products.

In [ ]:
def generate_counterfactuals_lr(model, sample_data, X_test_df, num_features, cat_features):
    """
    Generate counterfactual explanations using Logistic Regression model.
    Shows what changes would flip prediction from NOT RECOMMEND to RECOMMEND.
    """
    # Get prediction
    pred_proba = model.predict_proba(sample_data)[0][1]
    
    # Get original values
    original_values = sample_data.iloc[0]
    
    print(f"\nModel says: NOT RECOMMEND (Confidence: {(1-pred_proba)*100:.1f}%)")
    print(f"\nCurrent Product:")
    print(f"  - Category: {original_values['Division Name']} > {original_values['Department Name']} > {original_values['Class Name']}")
    print(f"  - Customer gave it: {original_values['Rating']:.1f} stars")
    print(f"  - Customer age: {original_values['Age']:.0f} years")
    print(f"  - Positive feedback: {original_values['Positive Feedback Count']:.0f}")
    
    # Search for counterfactuals
    counterfactuals = []
    np.random.seed(42)
    
    for attempt in range(300):
        new_sample = sample_data.copy()
        
        # Randomly modify one feature
        modify = np.random.choice(['division', 'department', 'class', 'rating', 'feedback'])
        
        if modify == 'division':
            divisions = X_test_df['Division Name'].unique()
            new_sample.iloc[0, new_sample.columns.get_loc('Division Name')] = np.random.choice(divisions)
        elif modify == 'department':
            departments = X_test_df['Department Name'].unique()
            new_sample.iloc[0, new_sample.columns.get_loc('Department Name')] = np.random.choice(departments)
        elif modify == 'class':
            classes = X_test_df['Class Name'].unique()
            new_sample.iloc[0, new_sample.columns.get_loc('Class Name')] = np.random.choice(classes)
        elif modify == 'rating':
            new_sample.iloc[0, new_sample.columns.get_loc('Rating')] = np.random.uniform(3.5, 5.0)
        elif modify == 'feedback':
            new_sample.iloc[0, new_sample.columns.get_loc('Positive Feedback Count')] = np.random.randint(5, 30)
        
        # Check if prediction flips
        new_pred_proba = model.predict_proba(new_sample)[0][1]
        
        if new_pred_proba > 0.5:
            new_values = new_sample.iloc[0]
            changes_desc = []
            
            if original_values['Division Name'] != new_values['Division Name']:
                changes_desc.append(f"Move to division '{new_values['Division Name']}'")
            if original_values['Department Name'] != new_values['Department Name']:
                changes_desc.append(f"Move to department '{new_values['Department Name']}'")
            if original_values['Class Name'] != new_values['Class Name']:
                changes_desc.append(f"Change class to '{new_values['Class Name']}'")
            if abs(original_values['Rating'] - new_values['Rating']) > 0.1:
                changes_desc.append(f"Improve quality from {original_values['Rating']:.1f} to {new_values['Rating']:.1f} stars")
            if abs(original_values['Positive Feedback Count'] - new_values['Positive Feedback Count']) > 1:
                changes_desc.append(f"Get more feedback (from {original_values['Positive Feedback Count']:.0f} to {new_values['Positive Feedback Count']:.0f})")
            
            if changes_desc and len(counterfactuals) < 3:
                counterfactuals.append({
                    'description': changes_desc,
                    'confidence': new_pred_proba
                })
        
        if len(counterfactuals) >= 3:
            break
    
    if counterfactuals:
        print(f"\n" + "="*80)
        print("TO MAKE CUSTOMERS RECOMMEND THIS PRODUCT, YOU CAN:")
        print("="*80)
        
        for i, cf in enumerate(counterfactuals, 1):
            print(f"\nOption {i}:")
            for change in cf['description']:
                print(f"  - {change}")
            print(f"  --> Result: {cf['confidence']*100:.1f}% chance of RECOMMENDATION")
    else:
        print("\nCould not find clear changes to improve recommendation.")
    
    print("\n" + "="*80 + "\n")

# Find products that were NOT recommended
X_test_df = X_test.reset_index(drop=True)
not_recommend_indices = np.where(y_pred == 0)[0]
print(f"Analyzing {len(not_recommend_indices)} products that were NOT recommended\n")

# Generate 2 examples
np.random.seed(42)
for i in range(2):
    print("="*80)
    print(f"EXAMPLE {i+1}")
    print("="*80)
    if i < len(not_recommend_indices):
        idx = np.random.choice(not_recommend_indices)
        sample = X_test_df.iloc[[idx]]
        generate_counterfactuals_lr(model, sample, X_test_df, numeric_features, categorical_features)
        not_recommend_indices = [x for x in not_recommend_indices if x != idx]

## 8. Final Summary

In [ ]:
print("\n" + "="*90)
print("FINAL SUMMARY")
print("="*90)
print(f"\nDataset:                      {df.shape[0]:,} reviews")
print(f"Model:                        Logistic Regression")
print(f"\nAccuracy:                     {accuracy*100:.2f}%")
print(f"Precision:                    {precision*100:.2f}%")
print(f"Recall:                       {recall*100:.2f}%")
print(f"F1-Score:                     {f1*100:.2f}%")
print(f"\nKey Insight:                  Logistic Regression performs excellently with simple features")
print(f"Main Factor:                  Product rating is the strongest predictor")
print(f"Typical Fix:                  Improve rating from 3.0 to 4.0+ stars")
print("="*90)